<a href="https://colab.research.google.com/github/Matendy12/Project-4/blob/main/Movies_and_Ratings_Unsupervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hvplot

In [ ]:
!pip install itables

In [ ]:
!pip install pandas

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'movies-and-ratings-for-recommendation-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3747724%2F6486174%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T214144Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3De42a0a0ca0318c3dd704c22d2f6ef7f60b1db72d51d5a17e44111857702a9b842e3f50e664305a06e60e05b027919a1aa5fd9875749204c5f1c595ab1a3f5966a0480145dd3880eb5ee6f6f3c723389106b95d816976137710830655a8a9791d6853ff17fb18513a7169ea4416acbced3d7da38f77bf81165e2c0ce6fed18f1e75af62f5e180997cb1eb94746439ffb007d2535f91b7c62b7ced3fafae73e4f118d802db3997e89cfb79dfb22e3b8825088516eec266dfa521018d1ccd3342a4de58fed99d398537f75feada93d7164ac54405e026c261850fd97eb2e43dfdc8ac0ef9288f729b0b3d615a5e0feed4e3045978a8f2ee8db216426bb5ee11e279'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 866276 bytes downloaded
Downloaded and uncompressed: movies-and-ratings-for-recommendation-system
Data source import complete.


In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from itables import init_notebook_mode
init_notebook_mode(all_interactive = True)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
movies = pd.read_csv("/kaggle/input/movies-and-ratings-for-recommendation-system/movies.csv", index_col="movieId")
ratings = pd.read_csv("/kaggle/input/movies-and-ratings-for-recommendation-system/ratings.csv")

print("Movie Dataset")
display(movies)
print("User Ratings Dataset")
display(ratings)

Movie Dataset


title  \
movieId                                              
1                                 Toy Story (1995)   
2                                   Jumanji (1995)   
3                          Grumpier Old Men (1995)   
4                         Waiting to Exhale (1995)   
5               Father of the Bride Part II (1995)   
...                                            ...   
193581   Black Butler: Book of the Atlantic (2017)   
193583                No Game No Life: Zero (2017)   
193585                                Flint (2017)   
193587         Bungo Stray Dogs: Dead Apple (2018)   
193609         Andrew Dice Clay: Dice Rules (1991)   

                                              genres  
movieId                                               
1        Adventure|Animation|Children|Comedy|Fantasy  
2                         Adventure|Children|Fantasy  
3                                     Comedy|Romance  
4                               Comedy|Drama|Romance  
5                                             Comedy  
...                                              ...  
193581               Action|Animation|Comedy|Fantasy  
193583                      Animation|Comedy|Fantasy  
193585                                         Drama  
193587                              Action|Animation  
193609                                        Comedy  

[9742 rows x 2 columns]

User Ratings Dataset


userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]

In [ ]:
movies.head()

title  \
movieId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                              genres  
movieId                                               
1        Adventure|Animation|Children|Comedy|Fantasy  
2                         Adventure|Children|Fantasy  
3                                     Comedy|Romance  
4                               Comedy|Drama|Romance  
5                                             Comedy

from matplotlib import pyplot as plt
import seaborn as sns
_df_24.groupby('title').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_25.groupby('genres').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['genres'].value_counts()
    for x_label, grp in _df_26.groupby('title')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('title')
_ = plt.ylabel('genres')

In [ ]:
movies = pd.DataFrame({'title': ['Movie 1', 'Movie 2', 'Movie 3'], 'revenue': [10000, 20000, 30000]})

In [ ]:
# Visualize a scatter plot of the data
movies.hvplot.scatter(x="title", y="revenue")

:Scatter   [title]   (revenue)

In [ ]:
# Start by importing the K-means algorithm
from sklearn.cluster import KMeans

In [ ]:
# Create and initialize the K-means model instance for 2 clusters
model = KMeans(n_clusters=2, random_state=1)

# Print the model
model

KMeans(n_clusters=2, random_state=1)

In [ ]:
import numpy as np
movies_array = movies.to_numpy()

In [ ]:
# Print the data types of the columns in the movies DataFrame
movies.dtypes

title      object
revenue     int64
dtype: object

In [ ]:
# Check for missing values in the movies DataFrame
movies.isnull().sum()

title      0
revenue    0
dtype: int64

In [ ]:
# Check the shape of the movies DataFrame
movies.shape

(3, 2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
movies_array[:, 0] = label_encoder.fit_transform(movies_array[:, 0])

In [ ]:
# Fit the data to the instance of the model
model.fit(movies_array)

KMeans(n_clusters=2, random_state=1)

In [ ]:
movies.drop(columns=["title"], inplace=True)

In [ ]:
print(model.n_features_in_)

2


In [ ]:
if movies.shape[1] != model.n_features_in_:
    print("The number of features in the data does not match the model's expectations.")

The number of features in the data does not match the model's expectations.


In [ ]:
!pip install scikit-learn
import sklearn.cluster as cluster

In [ ]:
model = cluster.KMeans(n_clusters=2)
model.fit(movies[["revenue"]])

KMeans(n_clusters=2)

In [ ]:
model = KMeans(n_clusters=2)

In [ ]:
model.fit(movies)

KMeans(n_clusters=2)

In [ ]:
# Make predictions about the data clusters using the trained model
movies_array = model.predict(movies)

In [ ]:
print(movies_array)

[0 0 1]


In [ ]:
ratings.head()

userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

from matplotlib import pyplot as plt
_df_12['movieId'].plot(kind='hist', bins=20, title='movieId')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_13['rating'].plot(kind='hist', bins=20, title='rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_14['timestamp'].plot(kind='hist', bins=20, title='timestamp')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_15.plot(kind='scatter', x='movieId', y='rating', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['rating']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'rating'}, axis=1)
              .sort_values('rating', ascending=True))
  xs = counted['rating']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_19.sort_values('rating', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rating')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['timestamp']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'timestamp'}, axis=1)
              .sort_values('timestamp', ascending=True))
  xs = counted['timestamp']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_20.sort_values('timestamp', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('timestamp')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_21['movieId'].plot(kind='line', figsize=(8, 4), title='movieId')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_22['rating'].plot(kind='line', figsize=(8, 4), title='rating')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_23['timestamp'].plot(kind='line', figsize=(8, 4), title='timestamp')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
ratings = ratings.reset_index(drop=True)

In [ ]:
movies.shape

(3, 1)

In [ ]:
ratings.shape

(100836, 4)

In [ ]:
print(movies.shape)

(3, 1)


In [ ]:
print(ratings.shape)

(100836, 4)


In [ ]:
print(ratings.iloc[:3].shape)

(3, 4)


In [ ]:
print(movies.dtypes)
print(ratings.dtypes)

revenue    int64
dtype: object
userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


In [ ]:
print(movies)
print(ratings)

   revenue
0    10000
1    20000
2    30000
        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]


In [ ]:
movies = movies.assign(rating=ratings.iloc[0])

In [ ]:
len(ratings.columns)

4

In [ ]:
# Create a new DataFrame with the ratings information
ratings_df = pd.DataFrame(ratings)

# Add the ratings DataFrame to the movies DataFrame
movies = pd.concat([movies, ratings_df], axis=1)

# Review the DataFrame
movies.head()

title  \
1                    Toy Story (1995)   
2                      Jumanji (1995)   
3             Grumpier Old Men (1995)   
4            Waiting to Exhale (1995)   
5  Father of the Bride Part II (1995)   

                                        genres  userId  movieId  rating  \
1  Adventure|Animation|Children|Comedy|Fantasy     1.0      3.0     4.0   
2                   Adventure|Children|Fantasy     1.0      6.0     4.0   
3                               Comedy|Romance     1.0     47.0     5.0   
4                         Comedy|Drama|Romance     1.0     50.0     5.0   
5                                       Comedy     1.0     70.0     3.0   

     timestamp  
1  964981247.0  
2  964982224.0  
3  964983815.0  
4  964982931.0  
5  964982400.0

from matplotlib import pyplot as plt
_df_39['movieId'].plot(kind='hist', bins=20, title='movieId')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_40['rating'].plot(kind='hist', bins=20, title='rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_41['timestamp'].plot(kind='hist', bins=20, title='timestamp')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_42.groupby('title').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['genres'].value_counts()
    for x_label, grp in _df_53.groupby('title')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('title')
_ = plt.ylabel('genres')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_54['title'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_54, x='movieId', y='title', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_55['genres'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_55, x='movieId', y='genres', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_56['title'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_56, x='rating', y='title', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_57['genres'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_57, x='rating', y='genres', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
ratings_df.head()

userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

from matplotlib import pyplot as plt
_df_27['movieId'].plot(kind='hist', bins=20, title='movieId')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_28['rating'].plot(kind='hist', bins=20, title='rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_29['timestamp'].plot(kind='hist', bins=20, title='timestamp')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_30.plot(kind='scatter', x='movieId', y='rating', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['rating']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'rating'}, axis=1)
              .sort_values('rating', ascending=True))
  xs = counted['rating']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_34.sort_values('rating', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('rating')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['timestamp']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'timestamp'}, axis=1)
              .sort_values('timestamp', ascending=True))
  xs = counted['timestamp']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_35.sort_values('timestamp', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('timestamp')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_36['movieId'].plot(kind='line', figsize=(8, 4), title='movieId')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_37['rating'].plot(kind='line', figsize=(8, 4), title='rating')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_38['timestamp'].plot(kind='line', figsize=(8, 4), title='timestamp')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
import holoviews as hv

In [ ]:
ratings_df.hvplot.scatter(
    x="userId",
    y="rating",
    by="movieId"
)

KeyboardInterrupt: 